In [3]:
import pandas as pd

# Battery Specifications (Tesla Powerwall 3)
BATTERY_CAPACITY_KWH = 13.5
MAX_CHARGE_RATE_KW = 5.8  # Continuous power rating
MAX_DISCHARGE_RATE_KW = 5.8  # Continuous power rating
ROUND_TRIP_EFFICIENCY = 0.90  # Estimated round-trip efficiency


def simulate_battery(data_filepath):
    """
    Simulates a home battery system using time-series data.

    Args:
        data_filepath: Path to the CSV file containing the time-series data.

    Returns:
        A pandas DataFrame with simulation results and the total savings/earnings.
    """

    # Load data
    df = pd.read_csv(data_filepath, parse_dates=[
                     'interval_start', 'interval_end'])
    # --- Sort data by time ---
    df.sort_values(by='interval_start', inplace=True)
    df.reset_index(drop=True, inplace=True)  # Reset index after sorting
    # Initialize battery state
    battery_charge_kwh = 0.0  # Start with an empty battery
    total_cost = 0.0
    total_earnings = 0.0

    # Store simulation results
    results = []

    # Iterate through the time-series data
    for index, row in df.iterrows():
        consumption_kwh = row['consumption']
        buy_rate = row['rate']
        # Handle cases where sell rate might be missing
        sell_rate = row['rate_sale'] if 'rate_sale' in row else 0

        # --- Battery State Machine Logic ---

        # 1. Can we charge the battery? (If there's a sell rate, we assume excess solar generation)
        if sell_rate > 0:
            potential_charge_kwh = MAX_CHARGE_RATE_KW * 0.5  # 30-minute interval
            charge_amount_kwh = min(
                potential_charge_kwh, BATTERY_CAPACITY_KWH - battery_charge_kwh)

            # Apply efficiency losses
            charge_amount_kwh *= ROUND_TRIP_EFFICIENCY

            battery_charge_kwh += charge_amount_kwh
            total_earnings += charge_amount_kwh * sell_rate * \
                (1/ROUND_TRIP_EFFICIENCY)  # Sell rate already considers the excess gen

        # 2. Should we discharge the battery to meet consumption?
        elif consumption_kwh > 0:
            potential_discharge_kwh = MAX_DISCHARGE_RATE_KW * 0.5
            discharge_amount_kwh = min(
                potential_discharge_kwh, battery_charge_kwh, consumption_kwh)

            battery_charge_kwh -= discharge_amount_kwh
            consumption_kwh -= discharge_amount_kwh
            total_cost -= discharge_amount_kwh * buy_rate

        # 3. Buy remaining energy needed from the grid if any
        if consumption_kwh > 0:
            total_cost += consumption_kwh * buy_rate

        # Store results for this interval
        results.append({
            'interval_start': row['interval_start'],
            'interval_end': row['interval_end'],
            'initial_battery_charge_kwh': battery_charge_kwh - (charge_amount_kwh if 'charge_amount_kwh' in locals() else 0) + (discharge_amount_kwh if 'discharge_amount_kwh' in locals() else 0),
            'consumption_kwh': row['consumption'],
            'buy_rate': buy_rate,
            'sell_rate': sell_rate,
            'charge_amount_kwh': charge_amount_kwh if 'charge_amount_kwh' in locals() else 0,
            'discharge_amount_kwh': discharge_amount_kwh if 'discharge_amount_kwh' in locals() else 0,
            'final_battery_charge_kwh': battery_charge_kwh,
            'cost_this_interval': (consumption_kwh * buy_rate if consumption_kwh > 0 else 0) - (discharge_amount_kwh * buy_rate if 'discharge_amount_kwh' in locals() else 0),
            'earnings_this_interval': charge_amount_kwh * sell_rate * (1/ROUND_TRIP_EFFICIENCY) if 'charge_amount_kwh' in locals() else 0,
        })

        # Reset charge and discharge for next iteration
        if 'charge_amount_kwh' in locals():
            del charge_amount_kwh
        if 'discharge_amount_kwh' in locals():
            del discharge_amount_kwh

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    # Calculate total savings and earnings from the entire simulation
    net_savings_earnings = total_earnings + total_cost  # Cost is negative savings

    return results_df, net_savings_earnings


# --- Example Usage ---
data_filepath = 'historic_data.csv'  # Replace with your data file
simulation_results, total_savings_earnings = simulate_battery(data_filepath)

# Print the results
print(simulation_results)
print(f"\nTotal Savings/Earnings from Battery: £{total_savings_earnings:.2f}")
simulation_results.to_csv('simulation_results.csv', index=False)
# You can further analyze the simulation_results DataFrame to
# understand battery behavior, charge/discharge patterns, etc.

                 interval_start               interval_end  \
0     2024-04-19 01:00:00+00:00  2024-04-19 02:30:00+01:00   
1     2024-04-19 01:30:00+00:00  2024-04-19 03:00:00+01:00   
2     2024-04-19 02:00:00+00:00  2024-04-19 03:30:00+01:00   
3     2024-04-19 02:30:00+00:00  2024-04-19 04:00:00+01:00   
4     2024-04-19 03:00:00+00:00  2024-04-19 04:30:00+01:00   
...                         ...                        ...   
13578 2025-01-26 22:00:00+00:00  2025-01-26 22:30:00+00:00   
13579 2025-01-26 22:30:00+00:00  2025-01-26 23:00:00+00:00   
13580 2025-01-26 23:00:00+00:00  2025-01-26 23:30:00+00:00   
13581 2025-01-26 23:30:00+00:00  2025-01-27 00:00:00+00:00   
13582 2025-01-27 00:00:00+00:00  2025-01-27 00:30:00+00:00   

       initial_battery_charge_kwh  consumption_kwh   buy_rate  sell_rate  \
0                             0.0            0.065   7.499940        NaN   
1                             0.0            0.072   7.499940        NaN   
2                          